## Performance Analysis

In this notebook, we will compare the performance of various neural network models, focusing on hyperparameter tuning and optimization strategies.

### Step 1: Visualize and Save Model Architecture

To start, we will visualize and save the architecture of our neural network models using TensorBoard. This will help us understand the structure of our models and monitor their training progress.

Here’s how we will proceed:

1. **Import TensorBoard**: Use the `torch.utils.tensorboard` module to set up TensorBoard.
2. **Create a Summary Writer**: Initialize a TensorBoard `SummaryWriter` to log the model architecture.
3. **Visualize the Model**: Pass a dummy input through the model and use TensorBoard to visualize the architecture.
4. **Save the Visualization**: Save the visualization to disk for later review.


In [5]:
#!pip install torchviz #Uncomment for the first time running the session on colab
!rm -rf Models1.py # Uncomment this line for Running over Collab
!wget https://raw.githubusercontent.com//hhosseinian/Face_Recognition/main/Models1.py # Uncomment this line for Running over Collab
import torch
import torch.nn as nn
from Models1 import Net
from torchviz import make_dot

# Assuming you have a model and some input
model = Net()
x = torch.randn(1, 1, 224, 224)  # Example input tensor

# Get the model output
y = model(x)

# Visualize the computational graph
dot = make_dot(y, params=dict(model.named_parameters()))
dot.render('model_architecture', format='png')  # Save as PNG file


--2024-08-08 03:54:19--  https://raw.githubusercontent.com//hhosseinian/Face_Recognition/main/Models1.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /hhosseinian/Face_Recognition/main/Models1.py [following]
--2024-08-08 03:54:19--  https://raw.githubusercontent.com/hhosseinian/Face_Recognition/main/Models1.py
Reusing existing connection to raw.githubusercontent.com:443.
HTTP request sent, awaiting response... 200 OK
Length: 3601 (3.5K) [text/plain]
Saving to: ‘Models1.py’

Models1.py          100%[===================>]   3.52K  --.-KB/s    in 0s      

2024-08-08 03:54:19 (43.9 MB/s) - ‘Models1.py’ saved [3601/3601]

Shape after conv layers: torch.Size([1, 256, 10, 10])
Shape after flattening: torch.Size([1, 25600])


ValueError: Expected more than 1 value per channel when training, got input size torch.Size([1, 2000])

In [ ]:
from torchviz import make_dot
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = nn.Conv2d(1, 32, 5)
        self.pool1 = nn.MaxPool2d(2,2)
        self.bn1 = nn.BatchNorm2d(32)

        self.conv2 = nn.Conv2d(32, 64, 5)
        self.pool2 = nn.MaxPool2d(2,2)
        self.bn2 = nn.BatchNorm2d(64)

        self.conv3 = nn.Conv2d(64, 128, 5)
        self.pool3 = nn.MaxPool2d(2,2)
        self.bn3 = nn.BatchNorm2d(128)

        self.conv4 = nn.Conv2d(128, 256, 5)
        self.pool4 = nn.MaxPool2d(2,2)
        self.bn4 = nn.BatchNorm2d(256)

        # To be calculated
        self.fc1 = nn.Linear(256 * 11 * 11, 2000)
        self.fc_norm1 = nn.BatchNorm1d(2000)
        self.fc2 = nn.Linear(2000, 1000)
        self.fc_norm2 = nn.BatchNorm1d(1000)
        self.fc3 = nn.Linear(1000, 136)

    def forward(self, x):
        x = F.relu(self.bn1(self.pool1(self.conv1(x))))
        x = F.relu(self.bn2(self.pool2(self.conv2(x))))
        x = F.relu(self.bn3(self.pool3(self.conv3(x))))
        x = F.relu(self.bn4(self.pool4(self.conv4(x))))

        x = x.view(x.size(0), -1)
        x = F.relu(self.fc_norm1(self.fc1(x)))
        x = F.relu(self.fc_norm2(self.fc2(x)))
        x = self.fc3(x)
        return x

# Create a dummy input tensor
dummy_input = torch.randn(1, 1, 244, 244)

# Initialize the model
model = Net()

# Perform a forward pass to calculate the flattened size
with torch.no_grad():
    x = dummy_input
    x = F.relu(model.bn1(model.pool1(model.conv1(x))))
    x = F.relu(model.bn2(model.pool2(model.conv2(x))))
    x = F.relu(model.bn3(model.pool3(model.conv3(x))))
    x = F.relu(model.bn4(model.pool4(model.conv4(x))))

    # Print the size of the flattened tensor
    flattened_size = x.numel()
    print(f"Flattened size: {flattened_size}")

# Update the model with the correct input size
model.fc1 = nn.Linear(flattened_size, 2000)

# Verify if the model works with TensorBoard
from torch.utils.tensorboard import SummaryWriter

# Set up TensorBoard SummaryWriter
writer = SummaryWriter()

# Visualize the model
try:
    writer.add_graph(model, dummy_input)
finally:
    # Close the writer
    writer.close()

print("Model architecture visualization saved to TensorBoard.")

# Generate the visualization
dot = make_dot(output, params=dict(model.named_parameters()))

# Save the visualization as PNG
dot.format = 'png'
dot.render(filename='/content/model_architecture')


Flattened size: 30976
Model architecture visualization saved to TensorBoard.


## Performance of the Network without Dropout Layers (Model1.py):
![Image](https://github.com/hhosseinian/Face_Recognition/blob/main/images/Model1_Performance_20Epoch.png?raw=1)
## Performance of the Network with Dropout Layers (Model3.py):
![Image](https://github.com/hhosseinian/Face_Recognition/blob/main/images/Model3_Performance_20Epoch.png?raw=1)
